In [169]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


database = "C:\\Projects\\SpringBoard\\SQL Case Study\\clubdb\\countryClub.db"
 
    # create a database connection
conn = create_connection(database)
with conn: 
    print("2. Query all tasks")
    select_all_tasks(conn)
 


2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [159]:
query1 = 'SELECT * FROM BOOKINGS'
data1 = conn.execute(query1)
bookings_df = pd.DataFrame(data1.fetchall())
bookings_df = bookings_df.rename(columns={0:'bookid', 1:"facid", 2:"memid", 3:"starttime", 4:'slots'})
bookings_df.head()


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [160]:
import pandas  as pd
query2 = 'SELECT * FROM FACILITIES'
data2 = conn.execute(query2)
facilities_df = pd.DataFrame(data2.fetchall())
facilities_df = facilities_df.rename(columns={0:'facid', 1:"name", 2:"membercost", 3:"guestcost", 4:'initialoutlay', 5:"monthlymaintenance"})
facilities_df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [161]:
query3 = 'SELECT * FROM MEMBERS'
data3 = conn.execute(query3)
members_df = pd.DataFrame(data3.fetchall())
members_df = members_df.rename(columns={0:'memid', 1:"surname", 2:"firstname", 3:"address", 4:'zipcode', 5:'telephone', 6:'recommendedby', 7:'joindate'})
members_df.head()


,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


part 2: Q10: 
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [189]:
query10 = 'SELECT f.name, sum (b.slots * (CASE WHEN b.memid = 0 THEN f.guestcost ELSE f.membercost END)) AS revenue \
    FROM bookings b \
    INNER JOIN facilities f ON b.facid = f.facid \
    group by f.name having revenue <1000'


In [190]:
data10 = conn.execute(query10)
df_q10 = pd.DataFrame(data10.fetchall())
df_q10.head()

,0,1
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180



Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [191]:
query11 = 'SELECT m2.firstname, m2.surname, m1.firstname as recommendes_firstname, m1.surname as recommendes_surname FROM Members AS m1  INNER JOIN members AS m2 ON m2.recommendedby = m1.memid'


In [192]:
data11 = conn.execute(query11)
df_q11 = pd.DataFrame(data11.fetchall())
df_q11.head()



,0,1,2,3
0,Janice,Joplette,Darren,Smith
1,Gerald,Butters,Darren,Smith
2,Nancy,Dare,Janice,Joplette
3,Tim,Boothe,Tim,Rownam
4,Ponder,Stibbons,Burton,Tracy


Q12: Find the facilities with their usage by member, but not guests

In [201]:
query12 = 'SELECT m.surname ,f.name, COUNT(1)AS usage FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid INNER JOIN Members as m on b.memid = m.memid WHERE b.memid !=0 GROUP BY m.surname, f.name;'

In [202]:
data12 = conn.execute(query12)
df_q12 = pd.DataFrame(data12.fetchall())
df_q12.head()


,0,1,2
0,Bader,Badminton Court,9
1,Bader,Massage Room 2,2
2,Bader,Pool Table,23
3,Bader,Snooker Table,33
4,Bader,Squash Court,2


Q13: Find the facilities usage by month, but not guests

In [210]:
query13 = 'SELECT f.name,strftime(\'%m\', b.starttime) as Month, COUNT(1) AS usage FROM Bookings AS b \
INNER JOIN Facilities AS f ON b.facid = f.facid \
WHERE b.memid !=0 \
GROUP BY f.name;'

SyntaxError: invalid syntax (<ipython-input-210-cd5fd2b7593d>, line 4)

In [212]:
query13 = 'SELECT f.name,strftime(\'%m\', b.starttime) as Month, COUNT(1) AS usage FROM Bookings AS b \
INNER JOIN Facilities AS f ON b.facid = f.facid \
WHERE b.memid !=0 \
GROUP BY Month;'

In [213]:
data13 = conn.execute(query13)
df_q13 = pd.DataFrame(data13.fetchall())
df_q13.head()

,0,1,2
0,Table Tennis,07,480
1,Tennis Court 1,08,1168
2,Tennis Court 1,09,1512
